In [5]:
# Save the books

import requests

# Get a request and save it as a txt file

books = {
    "863": "The Mysterious Affair at Styles",
    "1155": "The Secret Adversary",
    "58866": "The Murder on the Links",
    "61168": "The Man in the Brown Suit",
    "61262": "Poirot Investigates",
    "65238": "The Secret of Chimneys",
    "66446": "The Plymouth Express Affair",
    "67160": "The Hunter's Lodge Case",
    "67173": "The Missing Will",
    "69087": "The Murder of Roger Ackroyd",
    "70114": "The Big Four",
    "72824": "The Mystery of the Blue Train"
}

# Go through the dict

for id, title in books.items():
    res = requests.get(f'http://www.gutenberg.org/ebooks/{id}.txt.utf-8')
    # Write to txt file

    cleaned_text = res.text.encode('utf-8', errors='replace').decode('utf-8')
    
    filename = title.replace(" ", "-") + ".txt"
    with open(filename, 'w', encoding='utf-8') as file:
        # Clean res.text to have nothing that cannot be encoded in a .write() call
        file.write(cleaned_text)